In [9]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import messagebox
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpBinary

In [10]:
path = r'D:\Documentos\VSCODEs\Scripts\SOM\House_Rent_Dataset.csv'
df = pd.read_csv(path)


In [11]:
df = df.dropna()
df= df[['Posted On','Rent', 'BHK', 'Bathroom', 'Size', 'Floor', 'Area Type', 'City', 'Furnishing Status']]

def parse_floor(floor_str):
    if pd.isna(floor_str):
        return np.nan
    floor_str = floor_str.strip()
    if "Ground" in floor_str:
        return 0
    elif "Upper Basement" in floor_str:
        return -1
    elif "Lower Basement" in floor_str:
        return -2
    elif "out of" in floor_str:
        try:
            return int(floor_str.split(" out of")[0])
        except:
            return np.nan
    else:
        # Captura valores sueltos
        try:
            return int(floor_str)
        except:
            return np.nan

# Aplicar a la columna
df["Floor"] = df["Floor"].apply(parse_floor)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Posted On          4746 non-null   object
 1   Rent               4746 non-null   int64 
 2   BHK                4746 non-null   int64 
 3   Bathroom           4746 non-null   int64 
 4   Size               4746 non-null   int64 
 5   Floor              4746 non-null   int64 
 6   Area Type          4746 non-null   object
 7   City               4746 non-null   object
 8   Furnishing Status  4746 non-null   object
dtypes: int64(5), object(4)
memory usage: 333.8+ KB
None


# BIP problem

In [13]:
def BIP_model(client):
    model = LpProblem("Property_Selection", LpMaximize)
    x = [LpVariable(f"x_{i}", cat=LpBinary) for i in range(len(df))]

    model += lpSum(x)

    for i in range(len(df)):
        if df.iloc[i]["Rent"] > client["budget"]:
            model += x[i] == 0
        if df.iloc[i]["Size"] < client["min_size"]:
            model += x[i] == 0
        if client["city"] != "Any" and df.iloc[i]["City"] != client["city"]:
            model += x[i] == 0
        if client["area_type"] != "Any" and df.iloc[i]["Area Type"] != client["area_type"]:
            model += x[i] == 0
        if client["furnishing"] != "Any" and df.iloc[i]["Furnishing Status"] != client["furnishing"]:
            model += x[i] == 0
        if df.iloc[i]["Bathroom"] < client["bathrooms"]:
            model += x[i] == 0
        if df.iloc[i]["Floor"] < client["floors"]:
            model += x[i] == 0
        if df.iloc[i]["BHK"] < client["bedrooms"]:
            model += x[i] == 0

    model.solve()
    selected = [i for i in range(len(df)) if x[i].varValue == 1]
    return df.iloc[selected]

# Interfaz de usuario
def submit():
    client = {
        "budget": int(entry_budget.get()),
        "min_size": int(entry_size.get()),
        "city": city_var.get(),
        "area_type": area_var.get(),
        "furnishing": furnishing_var.get(),
        "bathrooms": int(entry_bath.get()),
        "floors": int(entry_floors.get()),
        "bedrooms": int(entry_bedrooms.get())
    }
    result = BIP_model(client)
    if result.empty:
        messagebox.showinfo("Resultados", "No hay propiedades que cumplan los criterios.")
    else:
        result.to_csv("propiedades_optimas.csv", index=False)
        messagebox.showinfo("Resultados", f"Se seleccionaron {len(result)} propiedades.\nGuardadas en propiedades_optimas.csv")

# Tkinter GUI
root = tk.Tk()
root.title("Simulador de Cliente")

tk.Label(root, text="Presupuesto mensual:").pack()
entry_budget = tk.Entry(root)
entry_budget.pack()

tk.Label(root, text="Tamaño mínimo (sqft):").pack()
entry_size = tk.Entry(root)
entry_size.pack()

tk.Label(root, text="Ciudad preferida:").pack()
city_var = tk.StringVar(root)
city_var.set("Any")
tk.OptionMenu(root, city_var, "Any", *df["City"].unique()).pack()

tk.Label(root, text="Tipo de área:").pack()
area_var = tk.StringVar(root)
area_var.set("Any")
tk.OptionMenu(root, area_var, "Any", *df["Area Type"].unique()).pack()

tk.Label(root, text="Estado de amueblado:").pack()
furnishing_var = tk.StringVar(root)
furnishing_var.set("Any")
tk.OptionMenu(root, furnishing_var, "Any", *df["Furnishing Status"].unique()).pack()

tk.Label(root, text="Número mínimo de baños:").pack()
entry_bath = tk.Entry(root)
entry_bath.pack()

tk.Label(root, text="Número mínimo de pisos:").pack()
entry_floors = tk.Entry(root)
entry_floors.pack()

tk.Label(root, text="Número mínimo de dormitorios:").pack()
entry_bedrooms = tk.Entry(root)
entry_bedrooms.pack()

tk.Button(root, text="Buscar propiedades", command=submit).pack()
root.mainloop()

# Sensitivity Analysis